In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nqDataLoader as nq #data loading library
from keras.preprocessing import sequence
# np.random.seed(0)
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
import tensorflow as tf
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# from keras.utils import to_categorical

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
## importing the early stage dataset 
early_stage = pd.read_csv('GT_DataPD_MIT-CS2PD.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
early_stage["gt"] = early_stage["gt"].astype(int)
early_stage.head()

,pID,gt,updrs108,afTap,sTap,nqScore,typingSpeed,file_1
0,1000,1,27,79.0,184.5,0.107179,56.866667,1424946827.1000_001_014.csv
1,1001,1,16,96.5,189.0,0.056286,118.000000,1427279751.1001_001_014.csv
2,1002,0,5,140.0,158.0,0.039519,119.037037,1426676689.1002_001_014.csv
3,1004,1,22,83.5,191.5,0.034853,74.266667,1429866367.1004_001_014.csv
4,1005,1,17,68.0,150.0,0.048307,74.969697,1430134526.1005_001_014.csv


In [5]:
hit_time_series = []
for index, row in early_stage.iterrows():
    fileloc = row.file_1
    keyPressed, htArr, pressArr, releaseArr =  nq.getDataFiltHelper( "data_MIT-CS2PD/" + early_stage.loc[index]['file_1'])
    htArr =np.array(htArr)
    hit_time_series.append(htArr)

X1 = hit_time_series 

In [6]:
## importing the de-novo dataset 
de_novo = pd.read_csv('GT_DataPD_MIT-CS1PD.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
print(len(de_novo))
de_novo["gt"] = de_novo["gt"].astype(int)
de_novo.head()

31


,pID,gt,updrs108,afTap,sTap,nqScore,typingSpeed,file_1,file_2
0,11,1,14.25,NaN,162.25,0.117543,189.372549,1402930351.011_001_014.csv,1403706430.011_003_014.csv
1,60,0,2.00,NaN,162.25,0.070350,60.533333,1402932300.060_001_014.csv,1403708258.060_003_014.csv
2,67,1,25.25,NaN,133.75,0.223411,54.333333,1401117235.067_001_014.csv,1401978395.067_003_014.csv
3,68,0,6.00,NaN,159.00,0.074973,71.800000,1401114972.068_001_014.csv,1401980765.068_003_014.csv
4,70,1,26.25,NaN,113.50,0.175751,39.614035,1404311419.070_001_014.csv,1404743687.070_003_014.csv


In [7]:
##using both files 
hit_time_series = []
for index, row in de_novo.iterrows():
    fileloc1 = row.file_1
    keyPressed, htArr1, pressArr, releaseArr =  nq.getDataFiltHelper( 'data_MIT-CS1PD/' + de_novo.loc[index]['file_1'])
    htArr1 = np.array(htArr1)
    keyPressed, htArr2, pressArr, releaseArr =  nq.getDataFiltHelper( 'data_MIT-CS1PD/' + de_novo.loc[index]['file_2'])
    htArr2 = np.array(htArr2)
    htArr =np.concatenate((htArr1,htArr2),axis =0)
    htArr=np.array(htArr)
    hit_time_series.append(htArr)
X2 = hit_time_series 

In [8]:
X = np.concatenate((X1,X2),axis=0)
y=  np.concatenate((early_stage['gt'],de_novo["gt"]),axis=0)
print(X.shape)
print(y)

(85,)
[1 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0
 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 1 0 0 1 1 0]


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [9]:
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
flat_list = list(itertools.chain(*X))
value = sum(flat_list) / len(flat_list)
print(value)
X_padded =sequence.pad_sequences(X,dtype='float32',padding='post',maxlen=6000,value =0)
sc = StandardScaler()
X_padded = sc.fit_transform(X_padded)
X_padded = np.reshape(X_padded,(85,6000,1))
# X_train,X_test,y_train,y_test = train_test_split(X_padded,y,test_size=0.17,random_state=0)


0.11633377817478088


In [10]:
from sklearn.model_selection import GridSearchCV
def create_model(n_conv_layers,n_filters,n_dense,dropout,kernel_size):
    model = Sequential()
    for i in range(int(n_conv_layers)):
        model.add(Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu', input_shape=(6000,1)))
        model.add(Dropout(dropout))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(n_dense, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
model = KerasClassifier(build_fn=create_model, verbose=1)
pipe = Pipeline([('model', model)])

In [12]:
# from sklearn.model_selection import StratifiedKFold
# param_grid={
#         'model__batch_size':[1], 
#         'model__epochs':[20],
#         'model__dropout':[0.5],
#         'model__n_conv_layers':[2],
#         'model__n_filters':[64,128],
#         'model__kernel_size':[2,3,5,11],
#         'model__n_dense' :[100]
           
#         }
# grid = GridSearchCV(estimator=pipe, param_grid=param_grid,pre_dispatch=7, cv=StratifiedKFold(5))
# grid_result = grid.fit(X_padded, y)
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


In [23]:
from sklearn.model_selection import cross_val_score
import keras
from sklearn.model_selection import StratifiedKFold
model2 = Sequential()
for i in range(int(2)):
    model2.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(6000,1)))
    model2.add(Dropout(0.5))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
model2.add(Dense(100, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
skf = skf.split(X_padded,y)

cv_score = []

for i, (train, test) in enumerate(skf):
    # currently keras doesn't have like model.reset(), so the easiest way
    # recompiling our model in every step of the loop see below more
    # create model
  

    print("Running Fold", i+1, "/", n_folds)
    model2.fit(X_padded[train],y[train] ,epochs=10, batch_size=1)
    result = model2.evaluate(X_padded[test], y[test])
    # if we want only the accuracy metric
    cv_score.append(result[1])
    keras.backend.clear_session()
print(np.mean(cv_score))
print("\nMean accuracy of the crossvalidation: {}".format(np.mean(cv_score),np.std(cv_score)))

Running Fold 1 / 5
Epoch 1/10
68/68 [==============================] - 2s 18ms/step - loss: 3.1292 - accuracy: 0.5000
Epoch 2/10
68/68 [==============================] - 1s 17ms/step - loss: 0.7052 - accuracy: 0.6618
Epoch 3/10
68/68 [==============================] - 1s 17ms/step - loss: 0.6551 - accuracy: 0.6471
Epoch 4/10
68/68 [==============================] - 1s 17ms/step - loss: 0.5251 - accuracy: 0.8088
Epoch 5/10
68/68 [==============================] - 1s 17ms/step - loss: 0.5325 - accuracy: 0.7794
Epoch 6/10
68/68 [==============================] - 1s 17ms/step - loss: 0.3042 - accuracy: 0.9265
Epoch 7/10
68/68 [==============================] - 1s 17ms/step - loss: 0.2054 - accuracy: 0.9559
Epoch 8/10
68/68 [==============================] - 1s 17ms/step - loss: 0.2118 - accuracy: 0.9412
Epoch 9/10
68/68 [==============================] - 1s 17ms/step - loss: 0.3769 - accuracy: 0.9118
Epoch 10/10
1/1 [==============================] - 0s 122ms/step - loss: 4.0765 - accuracy

In [24]:
cv_score


[0.4117647111415863, 1.0, 1.0, 1.0, 1.0]

In [ ]:
total_scores = [0.529411792755127, 1.0, 1.0, 0.8235294222831726, 1.0,0.4117647111415863, 1.0, 1.0, 1.0, 1.0]